# Atelier 5, PageRank et PageRank personnalisé #
Cet atelier porte sur l'algorithme PageRank et sa version personnalisée personnalisé pour effectuer des recommandations.  Utilisez la bibliothèque "igraph" pour cet exercice et les données de MovieLens 100k.

Chaque question compte pour 5 points.

## Question 1 ##
(1) Créez la matrice d'adjacence du graphe biparti que nous nommerons B.  Cette matrice doit avoir une dimension de (943+1682,943+1682).  Vous pouvez la créer à partir de la matrice d'adjacence des votes de dimension (943,1682).

In [2]:
import warnings

warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import tqdm.autonotebook as tqdm
import igraph as ig
import matplotlib.pyplot as plt

votes = pd.read_csv("data/votes.csv")
items = pd.read_csv("data/items.csv")
users = pd.read_csv("data/u.csv")

m = u_data.pivot(index="user.id", columns="item.id", values="rating").fillna(0)

## Question 2 ##
(2) Calculez le vecteur du PageRank de la matrice biparti B.

## Question 3 ##
(3) Les titres ci-dessous représentent les films de deux réalisateurs, Alfred Hitchcock et Stanley Kubrick, que l'on retrouve dans les données Movielens.  Utilisez la version personnalisée de l'algorithme PageRank et présumez un utilisateur qui aime les deux premiers de chaque liste (Birds, Vertigo, Full Metal [...] et Clockwork [...]), mais qui n'a pas de vote pour les autres.  Quels sont les 80 premier films qui seraient recommandés et comment cela se compare-t-il avec la version non personnalisée?

In [ ]:
hitchcock_titres = [
    "Birds, The (1963)",
    "Vertigo (1958)",
    "North by Northwest (1959)",
    "Rear Window (1954)",
    "Dial M for Murder (1954)",
    "Psycho (1960)",
]

kubrick_titres = [
    "Full Metal Jacket (1987)",
    "Clockwork Orange, A (1971)",
    "2001: A Space Odyssey (1968)",
    "Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)",
]

## Question 4 ##
(4) Comment pourrait-on intégrer à cet algorithme l'information du réalisateur de ces films? Quel est le résultat par rapport à (3)?